In [ ]:
from download_websites import download_websites
from generate_scraping_tasks import generate_scraping_tasks
from generate_solution_code import generate_solution_code
from extract_html_info import extract_relevant_information

import pandas as pd

In [4]:
websites_csv = 'websites.csv'
save_folder = 'downloaded_pages'
ROWS = (0, 0)
NUM_TASKS_PER_WEBSITE = 1

# Download websites
# download_websites(websites_csv, save_folder)

# Generate scraping tasks
websites = pd.read_csv(websites_csv)

for i, row in websites.iterrows():
    if i < ROWS[0] or i > ROWS[1]:
        continue

    category, website, link = row
    HTML_file = f'{save_folder}/{website}.html'
    HTML_elements = extract_relevant_information(HTML_file)
    with open(f'extracted_info/{website}.txt', 'w') as f:
        f.write(HTML_elements)

    generate_scraping_tasks(link, website, category, HTML_elements, NUM_TASKS_PER_WEBSITE)
    with open(f'scraping_tasks/{website}.txt', 'r') as f:
        scraping_tasks = pd.DataFrame([{'task': task} for task in f.readlines()])

    for i, task in scraping_tasks.iterrows():
        task = task['task']
        generate_solution_code(website, HTML_file, category, HTML_elements, task, i)

In [ ]:
import pandas as pd

# Load all csv files in folder
folder = "old/CSV_files/split_csv/"
all_csv = [folder + "websites_0.csv", folder + "websites_1.csv", folder + "websites_2.csv"]

# Concatenate all files in the list
df_from_each_file = (pd.read_csv(f) for f in all_csv)
df_merged = pd.concat(df_from_each_file, ignore_index=True)

# Dublicate second column
df_merged['Link'] = df_merged['Website']

# Remove .ending (fx .com .net etc) from website name
df_merged['Website'] = df_merged['Website'].str.replace('.com', '').str.replace('.net', '').str.replace('.org', '').str.replace('.uk', '').str.replace('.gov', '').str.replace('.mil', '').str.replace('.int', '')

# Save to csv
df_merged.to_csv('websites.csv', index=False)


In [ ]:
df = pd.read_csv('websites.csv')

# Shuffle rows
df = df.sample(frac=1).reset_index(drop=True)

# Save to csv
df.to_csv('websites.csv', index=False)